In [1]:
import pandas as pd
from collections import OrderedDict  # 리스트 중복 제거 & 순서 그대로 / 순서 뒤죽박죽 중복제거 : set
import copy
import time
import pickle
import re
import sys
import chardet
import warnings
warnings.filterwarnings(action='ignore') 

from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time 
import pandas as pd
import os
import time
import subprocess
import smtplib
import email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.encoders import encode_base64
from email.mime.image import MIMEImage
import time
import datetime

from selenium.common.exceptions import NoSuchElementException,WebDriverException,StaleElementReferenceException

# from bs4 import BeautifulSoup
# import schedule

----------

In [ ]:
## implicitly_wait(10) 설명 :
# 페이지이동 등의 명령어를 줬을 때 다음 웹페이지가 넘어올 때까지 기다리라는 뜻
# 드라이버를 열고 나서 한번만 설정해주면 됨
# 괄호 안에 숫자의 의미는 10초동안 웹페이지가 로딩될 때까지 기다리고 10초가 넘어가면 웹페이지가 로딩이 됐던 안됐던 다음 명령어를 실행하겠다는 것
# 10초면 어지간한 웹페이지가 로딩되는데 충분한 시간(상황에 적정한 시간을 설정하면 됨)
# 여기서 햇갈리지 말아야 할 것은 페이지가 넘어오기까지 최대 10초까지 기다리겠다는 의미이지
# 요청하고나서 무조건 10초를 기다리겠다는 것이 아님(무조건 10초를 기다리려면 time.sleep을 쓰면 됨)

In [2]:
nujuk_user_rank = []  # 유저 순위 누적 리스트
nujuk_user_nm = []  # 유저명 누적 리스트
nujuk_server_nm = []  # 유저 서버 누적 리스트
nujuk_job_nm = []  # 유저 직업 누적 리스트
nujuk_fame = []  # 유저 명성치 누적 리스트

In [4]:
driver = webdriver.Chrome(executable_path='C:/Users/user/Desktop/GitHub/Study/유저분석/chromedriver_win32/chromedriver')
driver.implicitly_wait(10)

In [5]:
# 순위는 순위 기준을 따르되, 캐릭터 순위는 직업,서버를 포함하지 않은 전체 던파 캐릭터에 대한 순위입니다.
start_tm = time.time()

# page_num : 몇번째 페이지까지 크롤링 할 것인지?
# for page_num in tqdm(range(1,101), desc="진행 상태 "):
for page_num in tqdm(range(1,3), desc="진행 상태 "):

    # 던파 랭킹 페이지 오픈 : 한 페이지당 30개의 랭킹 존재
    URL = 'https://dunfamoa.com/leaderboard/list?page=' + str(page_num)
    driver.get(url=URL)

    # 한 페이지에서 1번부터 30번까지 유저 정보 가져올 수 있음
    for i in range(1,31):
        # 유저 순위 누적 리스트
        user_rank = str(i + (30 * (page_num-1)))
        nujuk_user_rank.append(user_rank)
        # 유저명 누적 리스트
        user_nm_xpath = '//*[@id="Content"]/div/div/div/div[3]/div[' + str(i) + ']/div[3]/a[2]'
        id_input = driver.find_element_by_xpath(user_nm_xpath)
        nujuk_user_nm.append(id_input.text)
        # 유저 서버 누적 리스트
        server_nm_xpath = '//*[@id="Content"]/div/div/div/div[3]/div[' + str(i) + ']/div[3]/div[1]'
        id_input = driver.find_element_by_xpath(server_nm_xpath)
        nujuk_server_nm.append(id_input.text)
        # 유저 직업 누적 리스트
        job_nm_xpath = '//*[@id="Content"]/div/div/div/div[3]/div[' + str(i) + ']/div[3]/div[2]'
        id_input = driver.find_element_by_xpath(job_nm_xpath)
        nujuk_job_nm.append(id_input.text)
        # 유저 명성치 누적 리스트
        fame_xpath = '//*[@id="Content"]/div/div/div/div[3]/div[' + str(i) + ']/div[4]/div/div'
        id_input = driver.find_element_by_xpath(fame_xpath)
        nujuk_fame.append(id_input.text)
    # time.sleep(1)
    
print('소요시간 : ',time.time() - start_tm)

진행 상태 : 100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:18<00:00,  9.42s/it]

소요시간 :  18.9134738445282


In [ ]:
# 유저 순위 누적 리스트
nujuk_user_rank

In [ ]:
# 유저명 누적 리스트
nujuk_user_nm

In [ ]:
# 유저 서버 누적 리스트
nujuk_server_nm

In [ ]:
# 유저 직업 누적 리스트
nujuk_job_nm

In [ ]:
# 유저 명성치 누적 리스트
nujuk_fame

In [ ]:
# # 유저명 인코딩(user_nm은 현재 유니코드 / UTF-8 변환)
# encoded_nujuk_user_nm = []
# for user_nm in nujuk_user_nm:
#     encoded_string = str(user_nm.encode('utf-8'))
#     encoded_string = encoded_string.replace("b'","").replace("\\x","%").replace("'","")
#     encoded_nujuk_user_nm.append(encoded_string)

In [44]:
driver.close()

In [91]:
nujuk_user_rank = pd.DataFrame(nujuk_user_rank, columns = ['rank'])  # 유저 순위 누적 리스트
nujuk_user_nm = pd.DataFrame(nujuk_user_nm, columns = ['user_nm'])  # 유저명 누적 리스트
nujuk_server_nm = pd.DataFrame(nujuk_server_nm, columns = ['server_nm'])  # 유저 서버 누적 리스트
nujuk_job_nm = pd.DataFrame(nujuk_job_nm, columns = ['job_nm'])  # 유저 직업 누적 리스트
nujuk_fame = pd.DataFrame(nujuk_fame, columns = ['fame'])  # 유저 명성치 누적 리스트

In [92]:
data_df = pd.concat([nujuk_user_rank, nujuk_user_nm, nujuk_server_nm, nujuk_job_nm, nujuk_fame], axis = 1)
data_df.head(5)

,rank,user_nm,server_nm,job_nm,fame
0,1,빵문검귀이,카인,眞 검귀,"37,456"
1,2,大碩,카인,眞 크루세이더,"37,000"
2,3,斗魚TV이빵문,카인,眞 스트라이커,"36,723"
3,4,도적티비,카인,眞 쿠노이치,"36,719"
4,5,Malfurion,디레지에,眞 크루세이더,"36,535"


In [93]:
# 유저명/서버명/직업명 인코딩(UTF-8 변환)
data_df['encoded_user_nm'] = data_df['user_nm'].map(lambda x : str(x.encode('utf-8')).replace("b'","").replace("\\x","%").replace("'",""))
data_df['encoded_server_nm'] = data_df['server_nm'].map(lambda x : str(x.encode('utf-8')).replace("b'","").replace("\\x","%").replace("'",""))
data_df['encoded_job_nm'] = data_df['job_nm'].map(lambda x : str(x.encode('utf-8')).replace("b'","").replace("\\x","%").replace("'",""))

In [94]:
data_df.head(3)

,rank,user_nm,server_nm,job_nm,fame,encoded_user_nm,encoded_server_nm,encoded_job_nm
0,1,빵문검귀이,카인,眞 검귀,"37,456",%eb%b9%b5%eb%ac%b8%ea%b2%80%ea%b7%80%ec%9d%b4,%ec%b9%b4%ec%9d%b8,%e7%9c%9e %ea%b2%80%ea%b7%80
1,2,大碩,카인,眞 크루세이더,"37,000",%e5%a4%a7%e7%a2%a9,%ec%b9%b4%ec%9d%b8,%e7%9c%9e %ed%81%ac%eb%a3%a8%ec%84%b8%ec%9d%b4...
2,3,斗魚TV이빵문,카인,眞 스트라이커,"36,723",%e6%96%97%e9%ad%9aTV%ec%9d%b4%eb%b9%b5%eb%ac%b8,%ec%b9%b4%ec%9d%b8,%e7%9c%9e %ec%8a%a4%ed%8a%b8%eb%9d%bc%ec%9d%b4...


-------

# API 호출

In [140]:
# API key
my_api_key = 'JcefnKuRK19hN4bWnFiRpvucjd3zwRCa'

In [139]:
# 01.서버 정보
driver = webdriver.Chrome(executable_path='C:/Users/user/Desktop/GitHub/Study/유저분석/chromedriver_win32/chromedriver')
driver.implicitly_wait(10)

URL = 'https://api.neople.co.kr/df/servers?apikey=' + my_api_key
driver.get(url=URL)

xpath = '/html/body'
id_input = driver.find_element_by_xpath(xpath)
info_servers = id_input.text  # str 타입
info_servers = pd.DataFrame(info_servers.replace('"','').replace('{rows:[{','').replace('}]}','').split('},{'), columns = ['servers'])
info_servers['eng_servers'] = info_servers['servers'].map(lambda x : x.split(',')[0])
info_servers['kor_servers'] = info_servers['servers'].map(lambda x : x.split(',')[1])
info_servers['eng_servers'] = info_servers['eng_servers'].map(lambda x : x.split(':')[1])
info_servers['kor_servers'] = info_servers['kor_servers'].map(lambda x : x.split(':')[1])
info_servers = info_servers[['kor_servers','eng_servers']]
info_servers

,kor_servers,eng_servers
0,카인,cain
1,디레지에,diregie
2,시로코,siroco
3,프레이,prey
4,카시야스,casillas
5,힐더,hilder
6,안톤,anton
7,바칼,bakal


In [95]:
data_df = pd.merge(data_df, info_servers, left_on = 'server_nm', right_on = 'kor_servers', how = 'left')

In [100]:
data_df = data_df[['rank', 'user_nm', 'kor_servers', 'eng_servers', 'job_nm', 'fame', 'encoded_server_nm', 'encoded_job_nm', 'encoded_user_nm']]

In [103]:
# 삭제 ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ
data_df = data_df.iloc[:10]

In [105]:
data_df.head(3)

,rank,user_nm,kor_servers,eng_servers,job_nm,fame,encoded_server_nm,encoded_job_nm,encoded_user_nm
0,1,빵문검귀이,카인,cain,眞 검귀,"37,456",%ec%b9%b4%ec%9d%b8,%e7%9c%9e %ea%b2%80%ea%b7%80,%eb%b9%b5%eb%ac%b8%ea%b2%80%ea%b7%80%ec%9d%b4
1,2,大碩,카인,cain,眞 크루세이더,"37,000",%ec%b9%b4%ec%9d%b8,%e7%9c%9e %ed%81%ac%eb%a3%a8%ec%84%b8%ec%9d%b4...,%e5%a4%a7%e7%a2%a9
2,3,斗魚TV이빵문,카인,cain,眞 스트라이커,"36,723",%ec%b9%b4%ec%9d%b8,%e7%9c%9e %ec%8a%a4%ed%8a%b8%eb%9d%bc%ec%9d%b4...,%e6%96%97%e9%ad%9aTV%ec%9d%b4%eb%b9%b5%eb%ac%b8


In [167]:
# 02.캐릭터 검색
# for characterName, serverId in zip(data_df['encoded_user_nm'], data_df['encoded_server_nm']):
serverId = data_df['eng_servers'][0]
characterName = data_df['encoded_user_nm'][0]

URL = 'https://api.neople.co.kr/df/servers/' + serverId + '/characters?characterName=' + characterName + '&jobId=&jobGrowId=&limit=&wordType=&apikey=' + my_api_key
driver.get(url=URL)

xpath = '/html/body'
id_input = driver.find_element_by_xpath(xpath)
info_servers = id_input.text  # str 타입
info_servers = pd.DataFrame(info_servers.replace('"','').replace('{rows:[{','').replace('}]}','').split(','), columns = ['user_info'])
info_servers['col_nm'] = info_servers['user_info'].map(lambda x : x.split(':')[0])
info_servers['col_info'] = info_servers['user_info'].map(lambda x : x.split(':')[1])
info_servers = info_servers[['col_nm','col_info']]
info_servers

,col_nm,col_info
0,serverId,cain
1,characterId,bf6b24c203111c6ede8c2faf4be2b9d6
2,characterName,빵문검귀이
3,level,110
4,jobId,41f1cdc2ff58bb5fdc287be0db2a8df3
5,jobGrowId,92da05ec93fb43406e193ffb9a2a629b
6,jobName,귀검사(남)
7,jobGrowName,眞 검귀


In [168]:
info_servers

,col_nm,col_info
0,serverId,cain
1,characterId,bf6b24c203111c6ede8c2faf4be2b9d6
2,characterName,빵문검귀이
3,level,110
4,jobId,41f1cdc2ff58bb5fdc287be0db2a8df3
5,jobGrowId,92da05ec93fb43406e193ffb9a2a629b
6,jobName,귀검사(남)
7,jobGrowName,眞 검귀


In [142]:
# # 02.캐릭터 검색
# # wordType = 'match'  # 검색타입 : 동일 단어(match), 전문 검색(full)

# # for encoded_user_nm, encoded_server_nm in zip(data_df['encoded_user_nm'], data_df['encoded_server_nm']):
#     # print(encoded_user_nm, encoded_server_nm)

# serverId = data_df['eng_servers'][0]
# # serverId = 'cain'
# characterName = data_df['encoded_user_nm'][0]
# # characterName = 'Malfurion'

# URL = 'https://api.neople.co.kr/df/servers/' + serverId + '/characters?characterName=' + characterName + '&jobId=&jobGrowId=&limit=&wordType=&apikey=' + my_api_key
# # 직접했을떄 되는거 :
#       # 'https://api.neople.co.kr/df/servers/cain/characters?characterName=Malfurion&jobId=&jobGrowId=&limit=&wordType=&apikey=JcefnKuRK19hN4bWnFiRpvucjd3zwRCa'
# # URL = 'https://api.neople.co.kr/df/servers/cain/characters?characterName=Malfurion&jobId=<jobId>&jobGrowId=<jobGrowId>&limit=<limit>&wordType=<wordType>&apikey=JcefnKuRK19hN4bWnFiRpvucjd3zwRCa'
# driver.get(url=URL)

# # xpath = '/html/body'
# # id_input = driver.find_element_by_xpath(xpath)
# # info_servers = id_input.text
# # info_servers

In [130]:
serverId = data_df['eng_servers'][0]
serverId

'cain'

In [132]:
characterName = data_df['encoded_user_nm'][1]
characterName

'%e5%a4%a7%e7%a2%a9'

In [ ]:
ㄴㅇㅁㄴㅇㄴㅁㅇㅁㄴㅇㅁㄴ

In [127]:
driver = webdriver.Chrome(executable_path='C:/Users/user/Desktop/GitHub/Study/유저분석/chromedriver_win32/chromedriver')
driver.implicitly_wait(10)

In [129]:
# URL = 'https://api.neople.co.kr/df/servers?apikey=' + my_api_key
URL = 'https://api.neople.co.kr/df/servers/' + serverId + '/characters?characterName=' + characterName + '&jobId=<jobId>&jobGrowId=<jobGrowId>&limit=<limit>&wordType=<wordType>&apikey=' + my_api_key
driver.get(url=URL)

In [ ]:
xpath = '/html/body'
id_input = driver.find_element_by_xpath(xpath)

In [ ]:
## Xpath 시작
# 1차 클릭
xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div/span/span'
id_input=driver.find_element_by_xpath(xpath)
id_input.click()
time.sleep(1)

# 이름 입력 칸 클릭
xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input'
id_input=driver.find_element_by_xpath(xpath)
id_input.click()

# 휴대전화번호 입력
id_input.send_keys('010-6551-0340')

# 발화자 정보 입력
id_input.send_keys('공선호=공선호=010-6551-0340')
id_input.send_keys(Keys.ENTER)
id_input.send_keys('조아라=조아라=010-6863-5820')
time.sleep(1)

## 스크롤 끝까지 내리기
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [ ]:
# API 페이지 오픈
URL = 'https://developers.neople.co.kr/contents/apiDocs/df'
driver = webdriver.Chrome(executable_path='C:/Users/User/Desktop/Git_Hub/Study/유저분석/chromedriver_win32_98/chromedriver')
driver.implicitly_wait(3)
driver.get(url=URL)

# 로그인 클릭
xpath = '//*[@id="wrap"]/div[1]/div/a'
id_input = driver.find_element_by_xpath(xpath)
id_input.click()
time.sleep(1)

# 접속방법(구글 로그인) 클릭
xpath = '//*[@id="google"]/img'
id_input = driver.find_element_by_xpath(xpath)
id_input.click()
time.sleep(1)

## 셀레니움 창 전환
# 참고1 : https://wikidocs.net/82611
# 참고2 : https://jamanbbo.tistory.com/51

# 새로운 로그인 창으로 전환
# print(driver.window_handles)  # 셀레늄은 윈도우를 구분하기 위해 window_handles 변수에 이름을 리스트로 저장하고 있습니다. 이름은 셀레늄에 의해 임의로 생성된 중복되지 않는 값입니다. 리스트는 순서가 있는 자료구조이기 때문에 윈도우가 생성된 순서대로 저장돼 있겠죠? 0번 위치의 값이 메인 윈도우이며, 두 번째 값이 팝업된 윈도우를 의미합니다.
# driver.to_switch(driver.window_handles[1])  # 셀레늄이 바라보는 윈도우를 변경할 때는 to_switch 메서드를 사용합니다. 메서드에 window_handles에서 얻은 윈도우 이름을 전달합니다.
driver.switch_to_window(driver.window_handles[1])

xpath = '//*[@id="identifierId"]'
id_input = driver.find_element_by_xpath(xpath)
id_input.click()
time.sleep(1)

id_input.send_keys('fave5820@gmail.com')
id_input.send_keys(Keys.ENTER)

xpath = '//*[@id="next"]/div/button/span'
id_input = driver.find_element_by_xpath(xpath)
id_input.click()
time.sleep(1)


driver.get_window_position(driver.window_handles[1])  # # 이어서 웹 브라우져를 컨트롤하기 위해서는 메인 윈도우로 돌아오는 것을 잊으면 안됩니다.
driver.quit()  # 데이터를 모두 가져 왔다면 팝업창을 닫을 수 있습니다.
driver.to_switch( driver.window_handles[0] )  

In [ ]:
# LEFT_SHIFT 키를 놓지 않고 누른 상태로 ARROW_UP(방향키 위)키를 누름 
# ActionChains(driver).key_down(Keys.LEFT_SHIFT).key_down(Keys.ARROW_UP).key_down(Keys.LEFT_CONTROL).send_keys('C').key_up(Keys.CONTROL).perform()

In [ ]:
# 시작하면됨

# 1부터 9999까지임 : 10000가 끝번호임
# 오류나는 애들 : 365,495,702,497,873,875,1215,1255(빼먹었는지모름/들어간듯?)

try : 

    print(str(num)+' 번째 제출중')  # 현재 몇번째 대화문 보내는 중인지?

    org_kakao = nujuk_org_piece[num]
    mody_kakao = nujuk_mody_piece[num]

    ### 셀레니움 시작

    if num == 1289:
#         print('앙대')
#         URL = 'https://docs.google.com/forms/d/e/1FAIpQLScNwNCacI8PZGycKcHOxTyg4u9zOZIwkvYJvrmEWdlNBIB1fg/viewform' # 기존

        #크롬 드라이버 경로
        URL = 'https://docs.google.com/forms/d/e/1FAIpQLSc5p46y8r3q60UjbhUzOGK9br3qdetp81J1vwEyBlO7c-YuUA/viewform'
        driver = webdriver.Chrome(executable_path='C:/Users/User/Desktop/chromedriver_win32/chromedriver')
        driver.implicitly_wait(3)
        driver.get(url=URL)
    else:
#         print('좋았어')
        pass


    ## Xpath 시작
    # 1차 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 2차 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 3차 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 이름 입력 칸 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 이름 입력
    id_input.send_keys('공선호')

    # 휴대전화번호 칸 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div[1]/div/div[1]/input'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 휴대전화번호 입력
    id_input.send_keys('010-6551-0340')

    # 전자우편주소 칸 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[4]/div/div/div[2]/div/div[1]/div/div[1]/input'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 전자우편주소 입력
    id_input.send_keys('kongswrite@naver.com')

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 내보내기* 클릭    
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[2]/div/div/div[3]/div/div/span/div/div/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 대화 조각 원문 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[3]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 대화 조각 원문 입력
    id_input.send_keys(org_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 발화자 정보 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[4]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 발화자 정보 입력
    id_input.send_keys('공선호=공선호=010-6551-0340')
    id_input.send_keys(Keys.ENTER)
    id_input.send_keys('조아라=조아라=010-6863-5820')
    time.sleep(1)

    ## 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 발화자 간의 관계 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div/span/div/div[2]/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 비식별화 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[6]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 비식별화 대화 조각 입력
    id_input.send_keys(mody_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 제출 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 다른응답제출 클릭
    xpath = '/html/body/div[1]/div[2]/div[1]/div/div[4]/a'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

except WebDriverException :

    ## 이모지 에러
    print(str(num)+' 번째는 pass')  # 몇번째 대화문에 이모지가 있었는지?
    num = num + 1
    print(str(num)+' 번째 제출중')  # 현재 몇번째 대화문 보내는 중인지?

    org_kakao = nujuk_org_piece[num]
    mody_kakao = nujuk_mody_piece[num]

    # 대화 조각 원문 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[3]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 대화 조각 원문 입력
    id_input.send_keys(org_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 발화자 정보 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[4]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 발화자 정보 입력
    id_input.send_keys('공선호=공선호=010-6551-0340')
    id_input.send_keys(Keys.ENTER)
    id_input.send_keys('조아라=조아라=010-6863-5820')
    time.sleep(1)

    ## 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 발화자 간의 관계 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div/span/div/div[2]/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 비식별화 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[6]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 비식별화 대화 조각 입력
    id_input.send_keys(mody_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 제출 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 다른응답제출 클릭
    xpath = '/html/body/div[1]/div[2]/div[1]/div/div[4]/a'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()


except NoSuchElementException :

    ## 내보내기 에러1

    # 내보내기* 클릭    
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[2]/div/div/div[3]/div/div/span/div/div/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 대화 조각 원문 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[3]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 대화 조각 원문 입력
    id_input.send_keys(org_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 발화자 정보 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[4]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 발화자 정보 입력
    id_input.send_keys('공선호=공선호=010-6551-0340')
    id_input.send_keys(Keys.ENTER)
    id_input.send_keys('조아라=조아라=010-6863-5820')
    time.sleep(1)

    ## 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 발화자 간의 관계 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div/span/div/div[2]/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 비식별화 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[6]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 비식별화 대화 조각 입력
    id_input.send_keys(mody_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 제출 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 다른응답제출 클릭
    xpath = '/html/body/div[1]/div[2]/div[1]/div/div[4]/a'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()


except StaleElementReferenceException :

    ## 내보내기 에러2

    # 내보내기* 클릭    
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[2]/div/div/div[3]/div/div/span/div/div/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()
    time.sleep(1)

    # 대화 조각 원문 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[3]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 대화 조각 원문 입력
    id_input.send_keys(org_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 발화자 정보 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[4]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 발화자 정보 입력
    id_input.send_keys('공선호=공선호=010-6551-0340')
    id_input.send_keys(Keys.ENTER)
    id_input.send_keys('조아라=조아라=010-6863-5820')
    time.sleep(1)

    ## 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 발화자 간의 관계 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div/span/div/div[2]/label/div/div[1]/div/div[3]/div'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 비식별화 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[2]/div[6]/div/div/div[3]/div/div[1]/div[2]/textarea'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    ###########################################################################################################################
    # 비식별화 대화 조각 입력
    id_input.send_keys(mody_kakao)
    time.sleep(1)
    ###########################################################################################################################

    # 다음 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 제출 클릭
    xpath = '/html/body/div/div[2]/form/div[2]/div/div[3]/div[1]/div/div[2]/span/span'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()

    # 다른응답제출 클릭
    xpath = '/html/body/div[1]/div[2]/div[1]/div/div[4]/a'
    id_input=driver.find_element_by_xpath(xpath)
    id_input.click()


--------------------------------------------------------------------------------------------------